<a href="https://colab.research.google.com/github/marathomas/meerkat/blob/master/06_meerkat_hyb_fu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Classify" hyb/fu calls

## Prerequisites

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import sys
os.system('pip install "librosa==0.7.1"')
import librosa
import numpy as np
import statistics
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.legend import Legend
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc, gridspec
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import umap
from scipy.stats import zscore
from sklearn import metrics
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.legend import Legend
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc, gridspec
import matplotlib
import seaborn as sns
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
import scipy
from scipy.spatial.distance import euclidean as euclid
from scipy.spatial import distance_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


PROJECT_PATH = "/content/drive/My Drive/meerkat/" 
FIGURES = PROJECT_PATH+"figures/"
DATA = PROJECT_PATH+"data/"

os.chdir(PROJECT_PATH)

### Load dataset

On 03.08.2020, generated hyb/fu dataset:

In [3]:
df_fu_only = pd.read_pickle(DATA+'interactive_plot_df_fu_only') 

In [6]:
df_fu_only.head(4)

,Name,call_lable,indv,raw_audio,stretched_spectrograms,denoised_stretched_spectrograms,duration_s,certain_yn,UMAP1,UMAP2,UMAP3
27244,HYB [LEAD + CC],fu/hyb,HRT,"[0.0234375, 0.01953125, 0.0234375, 0.017578125...","[[-17.500807, -14.801298, -13.165972, -14.3510...","[[36.025368, 32.888, 29.391817, 24.40986, 21.8...",0.198,y,3.232423,1.133456,5.226632
27245,HYB [CC + LEAD],fu/hyb,HRT,"[0.005859375, 0.0078125, 0.013671875, 0.003906...","[[-12.528769, -13.270456, -14.6460285, -13.184...","[[40.552284, 30.489008, 26.989344, 23.931791, ...",0.159,y,3.949984,0.699307,5.525731
27246,HYB [CC + LEAD],fu/hyb,HRT,"[0.048828125, 0.029296875, 0.015625, 0.0097656...","[[-17.513077, -14.152397, -13.090355, -15.3907...","[[44.029728, 39.67316, 34.686623, 30.64796, 32...",0.150,y,4.047125,0.719069,5.459079
27247,HYB [CC + AGG],fu/hyb,HRT,"[0.048828125, 0.05859375, 0.044921875, 0.02148...","[[-35.253723, -37.56826, -22.705854, -21.93972...","[[25.261524, 16.750294, 26.19335, 25.529036, 1...",0.356,y,3.690878,0.065397,9.181638


--> missing information about focal/nonfocal, and denoised_spectrograms (unstretched). Re-do spectrograms based on raw audio, also re-extract focal vs. nonfocal information.

### Remove nonfocal

In [ ]:
lablenames = df_fu_only.Name.values
nf_list = ['nf', 'NF', 'NONFOC', 'nonfoc']

nonfocal_yn = ['y' if any(substring in y for substring in nf_list)==True else 'n' for y in lablenames]

In [88]:
pd.Series(nonfocal_yn).value_counts()

n    1032
y      92
dtype: int64

In [89]:
df_fu_only['nonfocal_yn'] = nonfocal_yn

In [90]:
df = df_fu_only.loc[df_fu_only['nonfocal_yn']=='n',:]
df.shape

(1032, 12)

In [91]:
df.head(3)

,Name,call_lable,indv,raw_audio,stretched_spectrograms,denoised_stretched_spectrograms,duration_s,certain_yn,UMAP1,UMAP2,UMAP3,nonfocal_yn
27244,HYB [LEAD + CC],fu/hyb,HRT,"[0.0234375, 0.01953125, 0.0234375, 0.017578125...","[[-17.500807, -14.801298, -13.165972, -14.3510...","[[36.025368, 32.888, 29.391817, 24.40986, 21.8...",0.198,y,3.232423,1.133456,5.226632,n
27245,HYB [CC + LEAD],fu/hyb,HRT,"[0.005859375, 0.0078125, 0.013671875, 0.003906...","[[-12.528769, -13.270456, -14.6460285, -13.184...","[[40.552284, 30.489008, 26.989344, 23.931791, ...",0.159,y,3.949984,0.699307,5.525731,n
27246,HYB [CC + LEAD],fu/hyb,HRT,"[0.048828125, 0.029296875, 0.015625, 0.0097656...","[[-17.513077, -14.152397, -13.090355, -15.3907...","[[44.029728, 39.67316, 34.686623, 30.64796, 32...",0.150,y,4.047125,0.719069,5.459079,n


### Infer samplerate

Shit, samplerate is missing! Again, dirty infer.

In [97]:
frames = [len(x) for x in df['raw_audio']]
df['samplerate_hz'] = frames/df['duration_s']

In [98]:
# Everything close to 8000 will go to 8000, everything close to 48000 to 48000

def adjust_samplerate(sr):
  diff_to_8000 = abs(8000-sr)
  diff_to_48000 = abs(48000-sr)

  if diff_to_8000<200:
    y = 8000
  elif diff_to_48000<200:
    y = 48000
  else:
    y=sr

  return y

In [99]:
true_sr = [adjust_samplerate(sr) for sr in df['samplerate_hz']]
pd.Series(true_sr).value_counts()

8000     952
48000     80
dtype: int64

In [100]:
df['samplerate_hz'] = true_sr

### Re-do spectrograms etc.

In [94]:
# Spectrogramming paramters

FFT_WIN = 0.03 # FFT_WIN*samplerate = length of fft/n_fft (number of audio frames that go in one fft)
FFT_HOP = FFT_WIN/8 # FFT_HOP*samplerate = n of audio frames between successive ffts
N_MELS = 40 # number of mel bins
WINDOW = 'hann' # each frame of audio is windowed by a window function (its length can also be
# determined and is then padded with zeros to match n_fft. we use window_length = length of fft
N_MFCC = 13 # Number of cepstral coefficients
FMAX = 4000


# Function that generates mel spectrogram from audio data
# Input: data: Audio data (np array of floats), 
#        rate: samplerate in Hz (integer)
#        n_mels: number of mel bands (integer)
#        window: spectrogram window generation type (String)
#        fft_win: window length in s (float)
#        fft_hop: hop between window start in s (float)
# Output: Spectrogram (np array of floats)

def generate_mel_spectrogram(data, rate, n_mels, window, fft_win , fft_hop):
    n_fft  = int(fft_win * rate) 
    hop_length = int(fft_hop * rate) 
        
    s = librosa.feature.melspectrogram(y = data ,
                                       sr = rate, 
                                       n_mels = n_mels , 
                                       fmax = FMAX, 
                                       n_fft = n_fft,
                                       hop_length = hop_length, 
                                       window = window, 
                                       win_length = n_fft)

    spectro = librosa.power_to_db(s, ref=np.max)

    return spectro

def generate_stretched_mel_spectrogram(data, sr, duration, n_mels, window, fft_win , fft_hop):
    n_fft  = int(fft_win * sr) 
    hop_length = int(fft_hop * sr) 
    stretch_rate = duration/MAX_DURATION

    D = librosa.stft(y=data, # spectrogramming
                     n_fft = n_fft,
                     hop_length = hop_length,
                     window=window,
                     win_length = n_fft
                     )
    
    D_stretched = librosa.core.phase_vocoder(D, stretch_rate, hop_length=hop_length) # stretch spectrogram
    D_stretched = librosa.power_to_db(np.abs(D_stretched)**2, ref=np.max)
    
    s = librosa.feature.melspectrogram(S=D_stretched,  # mel transform
                                       sr=sr,
                                       n_mels=n_mels,
                                       fmax=FMAX)
        
    #spectro = librosa.power_to_db(np.abs(s)**2, ref=np.max) # power to db transform

    return s

In [92]:
df = df.drop(columns=['stretched_spectrograms', 'denoised_stretched_spectrograms', 'UMAP1', 'UMAP2', 'UMAP3'])

In [101]:
spectrograms = df.apply(lambda row: generate_mel_spectrogram(row['raw_audio'], 
                                                             row['samplerate_hz'], 
                                                             N_MELS, 
                                                             WINDOW, 
                                                             FFT_WIN , 
                                                             FFT_HOP), 
                        axis=1)

df['spectrograms'] = spectrograms

In [102]:
denoised = [(spectrogram - np.median(spectrogram, axis=0)) for spectrogram in df['spectrograms']]
df['denoised_spectrograms'] = denoised

In [104]:
MAX_DURATION = 0.5

# Generate stretched spectrograms
spectrograms = df.apply(lambda row: generate_stretched_mel_spectrogram(row['raw_audio'],
                                                                       row['samplerate_hz'],
                                                                       row['duration_s'],
                                                                       N_MELS,
                                                                       WINDOW,
                                                                       FFT_WIN,
                                                                       FFT_HOP),
                        axis=1)

df['stretched_spectrograms'] = spectrograms

denoised_stretched = [(spectrogram - np.median(spectrogram, axis=0)) for spectrogram in df['stretched_spectrograms']]
df['denoised_stretched_spectrograms'] = denoised_stretched

In [107]:
df.to_pickle(DATA+'focal_hybfu_df')

### Merge datasets

In [109]:
df = pd.read_pickle(DATA+'df_denoised_balanced')
df = df[df.nonfocal_yn == 'n'] # for focal
df.reset_index(inplace=True)

df_hyb_fu = pd.read_pickle(DATA+'focal_hybfu_df')
df_hyb_fu['index_id'] = df_hyb_fu.index

In [112]:
df_hyb_fu['index_id'] = df_hyb_fu.index
df_hyb_fu.reset_index(inplace=True)

In [120]:
df_hyb_fu['hyb_fu_yn'] = ['y']*df_hyb_fu.shape[0]
df['hyb_fu_yn'] = ['n']*df.shape[0]

In [115]:
df = df[df_hyb_fu.columns]

In [123]:
df.head(4)

,index,Name,call_lable,indv,raw_audio,duration_s,certain_yn,nonfocal_yn,samplerate_hz,spectrograms,denoised_spectrograms,stretched_spectrograms,denoised_stretched_spectrograms,index_id,hyb_fu_yn
0,1598,ALARM,al,HMB,"[0.0, 0.001953125, 0.001953125, 0.0, -0.001953...",0.178,y,n,8000,"[[-7.209921, -5.1646175, -3.098959, -2.4054718...","[[25.639555, 26.477575, 27.715773, 28.758339, ...","[[-7.011978, -9.6945915, -19.352905, -4.998459...","[[42.728127, 36.027912, 21.339691, 33.3617, 33...",1598,n
1,1599,ALARM,al,HMB,"[-0.001953125, 0.0, -0.001953125, 0.001953125,...",0.317,y,n,8000,"[[-21.059677, -20.16883, -20.029144, -18.70754...","[[10.762169, 11.25013, 11.792789, 12.15715, 16...","[[-20.856384, -25.17736, -22.190731, -18.3026,...","[[31.449883, 24.311474, 19.03553, 20.888477, 1...",1599,n
2,1600,ALARM,al,HMB,"[0.005859375, 0.005859375, 0.0078125, 0.005859...",0.192,y,n,8000,"[[-13.698711, -14.600319, -16.215317, -15.7289...","[[24.188335, 22.856678, 18.059265, 18.200275, ...","[[-18.871624, -18.160267, -16.079659, -15.3467...","[[34.815525, 32.16957, 28.692219, 25.785461, 2...",1600,n
3,1601,ALARM,al,HMB,"[0.001953125, 0.00390625, 0.00390625, 0.003906...",0.134,y,n,8000,"[[-15.611752, -15.137981, -14.848536, -16.0097...","[[20.75996, 18.54282, 15.484863, 12.944105, 11...","[[-16.879604, -17.779907, -17.648483, -17.0208...","[[32.35832, 27.98457, 24.143658, 22.213814, 23...",1601,n


In [124]:
all = df.append(df_hyb_fu, ignore_index=True)

In [127]:
all.hyb_fu_yn.value_counts()

n    6687
y    1032
Name: hyb_fu_yn, dtype: int64

In [128]:
all.to_pickle(DATA+'df_focal_hyb_fu')

# Analysis

In [132]:
# Function that pads a spectrogram with zeros to a certain length
# Input: spectrogram (2D np array)
#        maximal length (Integer)
# Output: Padded spectrogram (2D np array)

def pad_spectro(spec,maxlen):
  padding = maxlen - spec.shape[1]
  z = np.zeros((spec.shape[0],padding))
  padded_spec=np.append(spec, z, axis=1)
  return padded_spec

## Load dataset

In [129]:
df = pd.read_pickle(DATA+'df_focal_hyb_fu')

## UMAP with all

In [131]:
specs = df.denoised_spectrograms.values 

In [133]:
maxlen= np.max([spec.shape[1] for spec in specs])
flattened_specs = [pad_spectro(spec, maxlen).flatten() for spec in specs]
specs_arr = np.asarray(flattened_specs)
data = specs_arr

# Normalize
data = StandardScaler().fit_transform(pd.DataFrame(data))

In [134]:
reducer = umap.UMAP(low_memory=True, n_components = 10, min_dist=0)

In [135]:
embedding = reducer.fit_transform(data)
embedding.shape # (27244, 2) all

(7719, 10)

In [143]:
x, y, z = embedding[:,0], embedding[:,1], embedding[:,2] #np.asarray(umap_df.duration_s.values) #embedding[:,2]
scat_labels = df.call_lable.values
labeltypes = sorted(list(set(scat_labels)))

labeltypes.remove('fu/hyb') # make these custom

# makes colors same as AVGN visualization
pal = sns.color_palette("Set2", n_colors=len(labeltypes))
#pal = sns.color_palette("deep", n_colors=len(labeltypes))
color_dict = dict(zip(labeltypes, pal))

color_dict['fu/hyb'] = (1,0,0) # make them red

c = [color_dict[val] for val in scat_labels]

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   #hovertext = umap_df.call_lable.values, #+ " (" +umap_df.Name.values+")",
                                   hovertext = df.indv.values+" "+df.Name.values,
                                   marker=dict(
                                       size=4,
                                       color=c,                # set color to an array/list of desired values
                                       #colorscale='Viridis',   # choose a colorscale
                                       opacity=0.8
                                       ))])

fig.update_layout(scene = dict(
                    xaxis_title='UMAP1',
                    yaxis_title='UMAP2',
                    zaxis_title='UMAP3'),
                    #zaxis_title='Duration (s)'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.show()

## "Classify" fu/hyb based on k nearest neighbor

In [184]:
from scipy.spatial.distance import euclidean as euclid

In [ ]:
# Only unambiguous calls

In [193]:
k=10

In [203]:
embedding_non_ambi = embedding[df['hyb_fu_yn']=='n',:]
embedding_ambi = embedding[df['hyb_fu_yn']=='y',:]

non_ambi_labels = df.loc[df['hyb_fu_yn']=='n',:].call_lable.values
ambi_labelnames = df.loc[df['hyb_fu_yn']=='y',:].Name.values
embedded_list = [embedding_non_ambi[i,:] for i in range(embedding_non_ambi.shape[0])]

In [207]:
neighbors = []
for i in range(embedding_ambi.shape[0]):
  x = embedding_ambi[i,:]
  distances = np.asarray([euclid(x,y) for y in embedded_list])
  neighbor_inds = np.argsort(distances)[:k]
  neighbor_labels = [non_ambi_labels[ind] for ind in neighbor_inds]
  neighbors.append(neighbor_labels)

In [210]:
print(ambi_labelnames[i], neighbors[i])

HYB [MOV+LD] ['mo', 'mo', 'mo', 'mo', 'mo']


To what percentage can the hyb/fu calls be assigned to either/or of the indicated labels by majority vote?

### Make list of hybfu labels

This is a PAIN to automate, because different wordings are used, sometimes there is a space between '+', sometimes not... But ok. There seems to be a systematic such that:

HYB [call1+call2]

fu call1+call2

* Remove all *,?,!, etc.
* Find out whether first word is hyb or fu
* Split everything before first space and everything after
* remove brackets in HYB
* remove all empty spaces
* split words based on +
* categorize labels according to the groups

In [219]:
import re

In [240]:
labelsep = re.compile('[\?\*\!\%\.\$]') # Match ? * ! %
clean_ambi_lables = [labelsep.sub('', i) for i in ambi_labelnames]

In [241]:
firstword = [i.split(' ')[0] for i in clean_ambi_lables]

In [242]:
pd.Series(firstword).value_counts()

fu     770
HYB    262
dtype: int64

In [243]:
clean_ambi_lables = [i.split(' ', 1)[1] for i in clean_ambi_lables]

In [244]:
labelsep = re.compile('[()[\]{}]') # Match brackets
clean_ambi_lables = [labelsep.sub('', i) for i in clean_ambi_lables]

In [255]:
faulty = [x if '+' not in x else x for x in clean_ambi_lables]
pd.Series(faulty).value_counts()

cc+agg       158
SN+CC        144
cc+agg        72
mov+s         61
s+mo          55
            ... 
mov+ld         1
MO + CC        1
mo+md          1
LEAD + CC      1
SOC+MO         1
Length: 102, dtype: int64

In [256]:
clean_ambi_lables = [x+'+'+x if '+' not in x else x for x in clean_ambi_lables]

In [291]:
first_label = [i.split('+', 1)[0] for i in clean_ambi_lables]
second_label = [i.split('+', 1)[1] for i in clean_ambi_lables]

In [292]:
# Remove all non-alphabaetical
regex = re.compile('[^a-zA-Z]')

first_label = [regex.sub('', i) for i in first_label]
second_label = [regex.sub('', i) for i in second_label]

In [285]:
pd.Series(first_label).value_counts()

cc       336
SN       178
s        147
mov      138
mo        70
soc       65
CC        55
MOV       17
ld         7
al         3
LD         3
MOVE       3
agg        3
MO         2
SOC        2
AGG        1
alarm      1
LEAD       1
dtype: int64

In [293]:
check = [lable if lable not in labels_to_calls.keys() else "recognized" for lable in first_label]
pd.Series(check).value_counts()

recognized    1032
dtype: int64

In [294]:
pd.Series(second_label).value_counts()

agg         276
CC          153
ld          134
s           117
soc          96
mo           69
cc           46
LD           38
AGG          25
mov          18
MOVE         13
al            9
SOC           8
MO            7
UNK           5
MOV           4
LEAD          4
SN            3
ALARM         2
md            1
c             1
alarm         1
aggoragg      1
cdeedrr       1
dtype: int64

In [295]:
check = [lable if lable not in labels_to_calls.keys() else "recognized" for lable in second_label]
pd.Series(check).value_counts()

recognized    1023
UNK              5
aggoragg         1
cdeedrr          1
c                1
md               1
dtype: int64

In [287]:
call_types = {
    'cc' :["cc","Marker", "Marque", "CC", "MARKER", "MARQUE"],
    'sn' :["sn","subm", "short","^s$", "s", "SN", "SUBM", "SHORT", "S"], 
    'mo' :["mo", "mov", "move", "MO","MOV","MOVE"],
    'agg':["ag", "agg", "aggress", "chat", "growl","AG","AGG","AGGRESS","CHAT","GROWL"],
    'ld' :["ld","LD","lead","LEAD"],
    'soc':["soc","SOCIAL", "so", "SOC", "social"],
    'al' :["al", "AL","ALARM", "alarm"]#,
    #'fu/hyb': ["fu", "FU", "Fu", 'HYB', 'hyb', 'Hyb']
    }
    
# make dictionary labels to calls
labels_to_calls = dict((v,k) for k in call_types for v in call_types[k])

In [297]:
first_label = [labels_to_calls[lable] if lable in labels_to_calls.keys() else "unknown" for lable in first_label]
second_label = [labels_to_calls[lable] if lable in labels_to_calls.keys() else "unknown" for lable in second_label]

In [299]:
pd.Series(second_label).value_counts()

agg        301
cc         199
ld         176
sn         120
mo         111
soc        104
al          12
unknown      9
dtype: int64

In [306]:
hybfu_calls = list(zip(firstword, first_label, second_label))

### Save dataframe

What percentage

In [310]:
hybfu_classi = pd.DataFrame(list(zip(ambi_labelnames, firstword, first_label, second_label)), columns =['Name', 'hybfu_type', 'first_label', 'second_label']) 

In [315]:
for i in range(k):
  hybfu_classi['neighbor_'+str(i+1)] = [neighbor[i] for neighbor in neighbors]

In [323]:
hybfu_classi['index_id'] = df_hyb_fu['index_id']

In [324]:
hybfu_classi.head(30)

,Name,hybfu_type,first_label,second_label,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5,index_id
0,HYB [LEAD + CC],HYB,ld,cc,soc,ld,ld,mo,soc,27244
1,HYB [CC + LEAD],HYB,cc,ld,soc,ld,cc,ld,soc,27245
2,HYB [CC + LEAD],HYB,cc,ld,soc,cc,cc,soc,ld,27246
3,HYB [CC + AGG],HYB,cc,agg,agg,agg,agg,agg,soc,27247
4,HYB (SN+CC),HYB,sn,cc,ld,ld,soc,soc,mo,27248
5,HYB (SN+CC),HYB,sn,cc,sn,sn,sn,ld,soc,27249
6,HYB (SN+CC),HYB,sn,cc,mo,sn,sn,sn,sn,27250
7,HYB (SN+CC),HYB,sn,cc,ld,ld,soc,soc,mo,27251
8,HYB (SN+CC),HYB,sn,cc,agg,soc,agg,agg,agg,27252
9,HYB (SN+CC),HYB,sn,cc,ld,ld,soc,mo,soc,27253


In [325]:
hybfu_classi.to_pickle(DATA+'hybfu_classi')

# Old stuff

In [ ]:
k=10

# Find k nearest neighbors
nbrs = NearestNeighbors(metric='euclidean',n_neighbors=k+1, algorithm='brute').fit(embed)
distances, indices = nbrs.kneighbors(embedding)

# need to remove the first neighbor, because that is the datapoint itself
indices = indices[:,1:]  
distances = distances[:,1:]

In [152]:
labels = df.call_lable
labelnames = df.Name

In [153]:
fu_hyb_indices = np.asarray(np.where(labels=='fu/hyb'))[0]

In [154]:
fu_hyb_indices = np.asarray(np.where(labels=='fu/hyb'))[0]

for i,ind in enumerate(fu_hyb_indices):
  nearest_neighbors = indices[ind]
  neighbor_labels = [labels[neighbor] for neighbor in nearest_neighbors]
  print(labelnames[ind], " : ", neighbor_labels)

HYB [LEAD + CC]  :  ['fu/hyb', 'soc', 'fu/hyb', 'ld', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb']
HYB [CC + LEAD]  :  ['soc', 'fu/hyb', 'fu/hyb', 'ld', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'cc']
HYB [CC + LEAD]  :  ['fu/hyb', 'soc', 'cc', 'cc', 'fu/hyb', 'fu/hyb', 'soc', 'fu/hyb', 'fu/hyb', 'ld']
HYB [CC + AGG]  :  ['agg', 'agg', 'agg', 'agg', 'soc', 'soc', 'agg', 'fu/hyb', 'soc', 'fu/hyb']
HYB (SN+CC)  :  ['fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb']
HYB (SN+CC)  :  ['sn', 'fu/hyb', 'sn', 'fu/hyb', 'sn', 'ld', 'soc', 'soc', 'soc', 'ld']
HYB (SN+CC)  :  ['mo', 'sn', 'fu/hyb', 'sn', 'sn', 'sn', 'sn', 'mo', 'mo', 'sn']
HYB (SN+CC)  :  ['fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb']
HYB (SN+CC)  :  ['agg', 'soc', 'agg', 'agg', 'agg', 'agg', 'agg', 'agg', 'soc', 'soc']
HYB (SN+CC)  :  ['fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 'fu/hyb', 

In [ ]:
fu_hyb_indices = np.asarray(np.where(df.call_lable=='fu/hyb'))[0]

nn_stats_dict = {}
calltypes = sorted(list(set(labels)))
labels = np.asarray(labels)

for calltype in calltypes:
  call_indices = np.asarray(np.where(labels==calltype))[0]
  calltype_counts = np.zeros((call_indices.shape[0],len(calltypes)))

  for i,ind in enumerate(call_indices):
    nearest_neighbors = indices[ind]
    for neighbor in nearest_neighbors:
      neighbor_label = labels[neighbor]
      calltype_counts[i,np.where(np.asarray(calltypes)==neighbor_label)[0][0]] += 1 
  nn_stats_dict[calltype] = calltype_counts